## modeling of estimation on sqft price

In [5]:
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1

import os,sys
import numpy as np
import pandas as pd

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), '../..', 'src')
sys.path.append(src_dir)

%aimport plot_utils
from plot_utils import *
%aimport preprocessing
from preprocessing import *
%aimport data_source
from data_source import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### DataSource 
* This module loads data from database
* Auto update views by executing database/preprocess.sql

In [6]:
ds = DataSource(host='localhost',port=5432, username='postgres', password='', database='sdra')
ds.show_views()
ds.update_views()

connect to database postgresql://postgres:@localhost:5432/sdra
['properties', 'property_features', 'transactions', 'property_addresses', 'property_address_transactions', 'property_estimate']
execute  c:\wenyan\dse_capstone\database\preprocess.sql


### Preprocess
* This module will deliver the valid dataset with finished features

In [7]:
df_view = ds.get_view_df('property_estimate')
pp = Preprocess(df_view)
df_all = pp.dataset()
df_all_valid = pp.dataset(feature='delivered', valid=True)

(1512560, 13)
converting view
converting pool
converting date
converting year_built
Index(['num_bath', 'date', 'pool', 'year_built', 'sqft', 'sqft_price',
       'sold_price', 'num_bed', 'view'],
      dtype='object')
converting sold_year
converting sold_age
converting view
converting pool
converting date


In [8]:
print(df_all.shape)
print(df_all_valid.shape)

(1512560, 11)
(1446992, 7)


### dataset for modeling
* suggest to use valid dataset - df_all_valid
* (not suggested) can also try df_all - all data with all features

In [9]:
# check the date of last transaction
print(df_all_valid['date'].max())

2017-10-24 00:00:00


In [10]:
# get only transactions in last one year
df_last_year = df_all_valid[(df_all_valid['date'] >= '2016-10-01') & (df_all_valid['date'] < '2017-10-01')]
df_last_year.shape

(37881, 7)

### regression model 
* target: sqft_price
* features: ...
* data set: last 1 year, 2 years, ...
* try different models